In [6]:
# Import necessary libraries
!pip install pypdf2 --quiet
import pandas as pd
import re
from PyPDF2 import PdfReader

# Function to extract text from a PDF
def extract_text_from_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text.replace('\n', ' ').strip()
    return text

# Function to parse tender details from the tender PDF
def extract_tender_details(tender_text):
    tender_details = {
        'Tender Title': re.search(r'Tender Title\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Tender Title\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Tender ID': re.search(r'Tender ID\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Tender ID\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Issued By': re.search(r'Issued By\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Issued By\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Description': re.search(r'Description\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Description\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Specifications': re.search(r'Specifications\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Specifications\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Additional Requirements': re.search(r'Additional Requirements\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Additional Requirements\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Selection Criteria': re.search(r'Selection Criteria\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Selection Criteria\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Submission Deadline': re.search(r'Submission Deadline\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Submission Deadline\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
        'Contact Email': re.search(r'Contact Email\s*:\s*(.+)', tender_text, re.IGNORECASE).group(1).strip() if re.search(r'Contact Email\s*:\s*(.+)', tender_text, re.IGNORECASE) else None,
    }
    return tender_details

# Function to parse bid details from the bid PDFs
def extract_bid_details(bid_text):
    bid_details = {
        'Bidder Name': re.search(r'Bidder Name\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Bidder Name\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Contact Email': re.search(r'Contact Email\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Contact Email\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Bid Submission Date': re.search(r'Bid Submission Date\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Bid Submission Date\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Price': re.search(r'Price\s*:\s*\$?(\d+(?:,\d{3})*(?:\.\d{2})?)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Price\s*:\s*\$?(\d+(?:,\d{3})*(?:\.\d{2})?)', bid_text, re.IGNORECASE) else None,
        'Delivery Time': re.search(r'Delivery Time\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Delivery Time\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Warranty': re.search(r'Warranty\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Warranty\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Specifications': re.search(r'Specifications\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Specifications\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
        'Additional Features': re.search(r'Additional Features\s*:\s*(.+)', bid_text, re.IGNORECASE).group(1).strip() if re.search(r'Additional Features\s*:\s*(.+)', bid_text, re.IGNORECASE) else None,
    }
    return bid_details

# Function to evaluate the best bid based on the tender criteria
def evaluate_best_bid(tender_details, bids_data):
    best_bid = None
    best_score = 0

    for bid in bids_data:
        score = 0

        # Check for price competitiveness (lower is better)
        if bid['Price']:
            try:
                price = float(bid['Price'].replace(',', '').replace('$', ''))
                # Example: compare with an estimated budget or tender specifications (adjust as needed)
                estimated_price = 500  # Replace with your logic to extract estimated price from tender_details
                if price <= estimated_price:  # Ensure your comparison is correct
                    score += 1
            except ValueError:
                pass  # Skip if conversion fails

        # Check for delivery time (shorter is better)
        if bid['Delivery Time']:
            try:
                delivery_time = int(re.search(r'(\d+)', bid['Delivery Time']).group(1))
                # Replace the following line with the actual tender delivery requirement extraction
                tender_delivery_time = 30
                if delivery_time <= tender_delivery_time:
                    score += 1
            except (ValueError, AttributeError):
                pass  # Skip if conversion fails or regex doesn't match

        # Check for warranty duration
        if bid['Warranty']:
            try:
                warranty_duration = int(re.search(r'(\d+)', bid['Warranty']).group(1))
                tender_warranty = 3  # Replace with the actual tender requirement
                if warranty_duration >= tender_warranty:
                    score += 1
            except (ValueError, AttributeError):
                pass  # Skip if conversion fails or regex doesn't match

        # Add additional criteria checks as needed

        # Update best bid if current bid has a higher score
        if score > best_score:
            best_score = score
            best_bid = bid

    return best_bid

# Main execution
tender_pdf_path = r"C:\Users\shuve\Desktop\Tender Details.pdf"
bid_pdf_paths = [r"C:\Users\shuve\Desktop\Tender Details.pdf",r"C:\Users\shuve\Desktop\Bid2.pdf",r"C:\Users\shuve\Desktop\Bid3.pdf"]

# Extract tender details
tender_text = extract_text_from_pdf(tender_pdf_path)
tender_details = extract_tender_details(tender_text)

# Extract bid details for each bid PDF
bids_data = []
for bid_path in bid_pdf_paths:
    bid_text = extract_text_from_pdf(bid_path)
    bid_details = extract_bid_details(bid_text)
    bids_data.append(bid_details)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Create DataFrames for display
df_tender = pd.DataFrame([tender_details])
df_bids = pd.DataFrame(bids_data)

# Evaluate the best bid based on tender criteria
best_bid = evaluate_best_bid(tender_details, bids_data)

# Display results
print("\nTender Details:")
df_tender


Tender Details:


,Tender Title,Tender ID,Issued By,Description,Specifications,Additional Requirements,Selection Criteria,Submission Deadline,Contact Email
0,Supply of Desktop Computers for Office Use T...,TND2024 -001 Issued By : TechProcure Solution...,TechProcure Solutions Pvt. Ltd. Description ...,Procurement of 50 desktop computers for use in...,• Processor : Intel Core i5 or higher. • RAM...,1. 3-year onsite warranty. 2. Delivery withi...,1. Price competitiveness. 2. Delivery timeli...,"February 29, 2024, 5:00 PM IST. Contact Emai...",tenders@techprocure.com


In [8]:
print("\nBids Details:")
df_bids


Bids Details:


,Bidder Name,Contact Email,Bid Submission Date,Price,Delivery Time,Warranty,Specifications,Additional Features
0,None,tenders@techprocure.com,None,None,None,None,• Processor : Intel Core i5 or higher. • RAM...,None
1,Beta Innovations Ltd. Contact Email : bids@b...,bids@betainnovations.com Bid Submission Date...,"February 26, 2024. Offer Details : • Price ...",720,30 days. • Warranty : 2 years onsite. Spec...,2 years onsite. Specifications : • Processo...,• Processor : Intel Core i7. • RAM : 16GB. ...,• Free installation and setup for all units.
2,Gamma Corp. Contact Email : gamma.corp@outlo...,gamma.corp@outlook.com Bid Submission Date :...,"February 28, 2024. Offer Details : • Price ...",740,20 days. • Warranty : 3 years onsite. Spec...,3 years onsite. Specifications : • Processo...,• Processor : Intel Core i5. • RAM : 16GB. ...,• Includes surge protector with each unit.


In [9]:

if best_bid:
    print("\nBest Bid Based on Tender Criteria:")
    print(best_bid)
else:
    print("\nNo suitable bid found based on the criteria.")

# # Optionally, save results to CSV
# df_tender.to_csv('/content/tender_details.csv', index=False)
# df_bids.to_csv('/content/bids_details.csv', index=False)

# print("\nTender and bids details have been saved to CSV files.")


Best Bid Based on Tender Criteria:
{'Bidder Name': 'Gamma Corp.   Contact Email : gamma.corp@outlook.com   Bid Submission Date : February 28, 2024.   Offer Details :  • Price : $740 per unit.   • Delivery Time : 20 days.   • Warranty : 3 years onsite.   Specifications :  • Processor : Intel Core i5.   • RAM : 16GB.   • Storage : 512GB SSD.   • Operating System : Windows 11 Professional.   • Display : 21.5 -inch Full HD.   Additional Features :  • Includes surge protector with each unit.', 'Contact Email': 'gamma.corp@outlook.com   Bid Submission Date : February 28, 2024.   Offer Details :  • Price : $740 per unit.   • Delivery Time : 20 days.   • Warranty : 3 years onsite.   Specifications :  • Processor : Intel Core i5.   • RAM : 16GB.   • Storage : 512GB SSD.   • Operating System : Windows 11 Professional.   • Display : 21.5 -inch Full HD.   Additional Features :  • Includes surge protector with each unit.', 'Bid Submission Date': 'February 28, 2024.   Offer Details :  • Price : $74